In [10]:
import re
import pandas as pd
import numpy as np
import seaborn as sns
#import emoji
#from collections import Counter
import matplotlib.pyplot as plt
#from PIL import Image
#from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

# definition of functions

### extract the date time

In [2]:
def date_time(s) :
    pattern = '^([0-9]+)(\/)([0-9]+)(\/)([0-9]+), ([0-9]+):([0-9]+)[ ]?(AM|PM|am|pm)? -'
    result = re.match(pattern, s)
    if result :
        return True
    else :
        return False

### Extract the contacts

In [3]:
def find_contact(s) :
    s = s.split(":")
    if len(s) == 2 :
        return True
    else :
        return False

### find the message

In [4]:
def getMessage(line) :
    splitline = line.split(' - ')
    datetime = splitline[0]
    date, time = datetime.split(', ')
    message = " ".join(splitline[1:])
    if find_contact(message) :
        splitMesssage = message.split(": ")
        author = splitMesssage[0]
        message = " ".join(splitline[1:])
    else :
        author = None
    return date, time, author, message

In [11]:
data_conv = []
conversation = 'chat between two users.txt'
with open(conversation, encoding = "utf-8") as fp :
    fp.readline()
    messageBuffer = []
    date, time, author = None, None, None
    while True :
        line = fp.readline()
        if not line :
            break
        line = line.strip()
        if date_time(line) :
            if len(messageBuffer) > 0 :
                data_conv.append([date, time, author, ''.join(messageBuffer)])
            messageBuffer.clear()
            date, time, author, message = getMessage(line)
            messageBuffer.append(message)
        else :
            messageBuffer.append(line)

### analyze the sentiments

In [13]:
df = pd.DataFrame(data_conv, columns = ["Date", "Time", "Contact", "Message"])
df['Date'] = pd.to_datetime(df['Date'])
data_conv = df.dropna()

In [14]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sentiments = SentimentIntensityAnalyzer()
data_conv['positive'] = [sentiments.polarity_scores(i)["pos"] for i in data_conv["Message"]]
data_conv['negative'] = [sentiments.polarity_scores(i)["neg"] for i in data_conv["Message"]]
data_conv['neutral'] = [sentiments.polarity_scores(i)["neu"] for i in data_conv["Message"]]

data_conv.head(20)

,Date,Time,Contact,Message,positive,negative,neutral
0,2023-04-05,01:25,Aindrila Basu,Aindrila Basu: How are you?,0.000,0.000,1.000
1,2023-04-05,01:25,Swastik,Swastik: I am fine.,0.474,0.000,0.526
2,2023-04-05,01:25,Swastik,Swastik: Did you eat ?,0.000,0.000,1.000
3,2023-04-05,01:26,Aindrila Basu,Aindrila Basu: Yes I had my dinner.,0.351,0.000,0.649
4,2023-04-05,01:26,Swastik,Swastik: How was the pizza ?,0.000,0.000,1.000
5,2023-04-05,01:26,Aindrila Basu,Aindrila Basu: The pizza was not good.,0.000,0.286,0.714
6,2023-04-05,01:27,Aindrila Basu,Aindrila Basu: How was your day?,0.000,0.000,1.000
7,2023-04-05,01:27,Swastik,Swastik: The rain made me sad,0.000,0.383,0.617
8,2023-04-05,01:27,Swastik,Swastik: what about you ?,0.000,0.000,1.000
9,2023-04-05,01:28,Aindrila Basu,"Aindrila Basu: I agree, I don't like rain either.",0.260,0.220,0.520


In [8]:
x = sum(data['positive'])
y = sum(data['negative'])
z = sum(data['neutral'])

def score(a, b, c) :
    if a > b and a > c :
        print("Positive")
    if b > a and b > c :
        print("Negative")
    if c > a and c > b :
        print("Neutral")

score(x, y, z)

Neutral
